In [1]:
!pip install pykeen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 20.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia

In [2]:
import pykeen
from pykeen.regularizers import LpRegularizer
from pykeen.models import TransE, RotatE, TuckER
from pykeen.optimizers import Adam
from pykeen.losses import MarginRankingLoss, BCEAfterSigmoidLoss
from pykeen.training import SLCWATrainingLoop
from pykeen.sampling import BernoulliNegativeSampler
from pykeen.evaluation import RankBasedEvaluator
from pykeen.pipeline import pipeline
from pykeen.hpo import hpo_pipeline, HpoPipelineResult
from pykeen.triples import TriplesFactory
from optuna.pruners import MedianPruner
from optuna.storages import RDBStorage
from pykeen.stoppers import EarlyStopper
from pykeen.checkpoints import save_model
import torch
import optuna
import json

In [3]:
data=TriplesFactory.from_path('/kaggle/input/dataset-ml/triples.tsv')
complete_train, test=data.split([0.8,0.2], random_state=42)
train,val=complete_train.split([0.8,0.2], random_state=42)

# Hyperparameter optimization

## TransE

In [5]:
storage = RDBStorage("sqlite:////kaggle/working/TransE_optuna_study.db")

optimization_transE=hpo_pipeline(
    storage=storage,
    study_name='TransE_hpo_study_',
    load_if_exists=True,
    n_trials=25,
    training_kwargs=dict(num_epochs=50, batch_size=512),
    training=train,
    validation=val,
    testing=test,
    model='transe',

    model_kwargs_ranges=dict(
        embedding_dim=dict(type=int, low=100, high=400, q=100),
        entity_initializer=dict(type='categorical', choices=[
            'xavier_uniform',
            'xavier_uniform_norm',
    ]),
    scoring_fct_norm=dict(type=float, low=1.0, high=2.0, q=0.5)),

    optimizer='adam',
    optimizer_kwargs_ranges=dict(lr=dict(type='categorical', choices=[1e-5, 1e-4, 1e-3])),

    loss='MarginRankingLoss',
    loss_kwargs_ranges=dict(
        margin=dict(type=float, low=1, high=10, q=1),
    ),

    training_loop='slcwa',
    regularizer='lpregularizer',
    regularizer_kwargs_ranges=dict(
        p=dict(type=int, low=1, high=2),
        weight=dict(type='categorical', choices=[1e-5, 1e-4, 1e-3, 1e-2])
    ),

    negative_sampler='bernoulli',
    negative_sampler_kwargs=dict(
        filtered=True,
    ),
    negative_sampler_kwargs_ranges=dict(
        num_negs_per_pos=dict(type=int, low=2, high=14, q=2)),

    evaluator='RankBasedEvaluator',
    evaluator_kwargs=dict(filtered=True),
    evaluation_kwargs=dict(batch_size=256),
    pruner = MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=10,
        interval_steps=5
        ),
    
    device='cuda'
)

optimization_transE.save_to_directory('/kaggle/working/hpo_results')
print('study_saved')

[I 2025-07-14 13:24:02,677] Using an existing study with name 'TransE_hpo_study_' instead of creating a new one.


Training epochs on cuda:0:   0%|          | 0/50 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

...

[I 2025-07-14 13:36:18,710] Trial 25 finished with value: 0.47365280985832214 and parameters: {'model.embedding_dim': 300, 'model.scoring_fct_norm': 1.0, 'model.entity_initializer': 'xavier_uniform_norm', 'loss.margin': 7.0, 'regularizer.weight': 1e-05, 'regularizer.p': 1, 'optimizer.lr': 0.001, 'negative_sampler.num_negs_per_pos': 8}. Best is trial 0 with value: 0.47675469517707825.


Training epochs on cuda:0:   0%|          | 0/50 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

[W 2025-07-14 13:36:25,714] Trial 26 failed with parameters: {'model.embedding_dim': 300, 'model.scoring_fct_norm': 1.0, 'model.entity_initializer': 'xavier_uniform_norm', 'loss.margin': 7.0, 'regularizer.weight': 1e-05, 'regularizer.p': 1, 'optimizer.lr': 0.0001, 'negative_sampler.num_negs_per_pos': 10} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pykeen/hpo/hpo.py", line 267, in __call__
    result = pipeline(
             ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pykeen/pipeline/api.py", line 1540, in pipeline
    stopper_instance, configuration, losses, train_seconds = _handle_training(
                                                             ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/

KeyboardInterrupt: 

## RotatE

In [ ]:
storage = RDBStorage("sqlite:////kaggle/working/optuna_study_RotatE2.db")
try:
    optimization_rotate=hpo_pipeline(
        storage=storage,
        study_name='RotatE_hpo_study_',
        load_if_exists=True,
        n_trials=25,
        training_kwargs=dict(num_epochs=50, batch_size=512),
        training=train,
        validation=val,
        testing=test,
        model='rotate',

        model_kwargs_ranges=dict(
            embedding_dim=dict(type=int, low=100, high=400, q=100),
            entity_initializer=dict(type='categorical', choices=[
                'xavier_uniform',
                'xavier_uniform_norm',
        ])),

        optimizer='adam',
        optimizer_kwargs_ranges=dict(lr=dict(type='categorical', choices=[1e-5, 1e-4, 1e-2])),

        loss='MarginRankingLoss',
        loss_kwargs_ranges=dict(
            margin=dict(type=float, low=1, high=5, q=1),
        ),

        training_loop='slcwa',
        regularizer='lpregularizer',
        regularizer_kwargs_ranges=dict(
        p=dict(type=int, low=1, high=2),
        weight=dict(type='categorical', choices=[1e-5, 1e-4, 1e-3, 1e-2])
        ),

        negative_sampler='bernoulli',
        negative_sampler_kwargs=dict(
            filtered=True,
        ),
        negative_sampler_kwargs_ranges=dict(
            num_negs_per_pos=dict(type=int, low=2, high=14, q=2)),

        evaluator='RankBasedEvaluator',
        evaluator_kwargs=dict(filtered=True),
        evaluation_kwargs=dict(batch_size=256),
    
        pruner = MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=10,
        interval_steps=5
        ),

        device='cuda'
)

## TuckER

In [ ]:
storage = RDBStorage("sqlite:////kaggle/working/optuna_study_TuckER.db")
try:
    optimization_rotate=hpo_pipeline(
        storage=storage,
        study_name='TuckER_hpo_study',
        load_if_exists=True,
        n_trials=25,
        training_kwargs=dict(num_epochs=50, batch_size=512),
        training=train,
        validation=val,
        testing=test,
        model='tucker',

        model_kwargs_ranges=dict(
            embedding_dim=dict(type=int, low=100, high=400, q=100),
            dropout_0=dict(type=float, low=0.1, high=0.3, q=0.1),
            dropout_1=dict(type=float, low=0.1, high=0.3, q=0.1),
            dropout_2=dict(type=float, low=0.1, high=0.4, q=0.1),
            entity_initializer=dict(type='categorical', choices=[
                'xavier_uniform',
                'xavier_uniform_norm',
        ])),

        optimizer='adam',
        optimizer_kwargs_ranges=dict(lr=dict(type='categorical', choices=[5e-4, 1e-3, 1e-2])),

        loss='BCEAfterSigmoidLoss',

        training_loop='slcwa',

        negative_sampler='bernoulli',
        negative_sampler_kwargs=dict(
            filtered=True,
        ),
        negative_sampler_kwargs_ranges=dict(
            num_negs_per_pos=dict(type=int, low=2, high=14, q=2)),

        evaluator='RankBasedEvaluator',
        evaluator_kwargs=dict(filtered=True),
        evaluation_kwargs=dict(batch_size=256),
    
        pruner = MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=10,
        interval_steps=5
        ),

        device='cuda'
)

In [4]:
#Loading TransE study
TransE_study = optuna.load_study(
    study_name="TransE_hpo_study_",
    storage='sqlite://///kaggle/working/TransE_optuna_study.db',
)

#Loading RotatE study
RotatE_study = optuna.load_study(
    study_name="RotatE_hpo_study_",
    storage='sqlite://///kaggle/working/optuna_study_RotatE2.db',
)

#Loading TuckER study
Tucker_study = optuna.load_study(
    study_name="TuckER_hpo_study",
    storage='sqlite://///kaggle/working/optuna_study_TuckER.db',
)

In [5]:
TransE_best_params=TransE_study.best_params
print(TransE_best_params)
RotatE_best_params=RotatE_study.best_params
print(RotatE_best_params)
Tucker_best_params=Tucker_study.best_params
print(Tucker_best_params)

{'model.embedding_dim': 400, 'model.scoring_fct_norm': 1.5, 'model.entity_initializer': 'xavier_uniform_norm', 'loss.margin': 3.0, 'regularizer.weight': 1e-05, 'regularizer.p': 1, 'optimizer.lr': 0.0001, 'negative_sampler.num_negs_per_pos': 12}
{'model.embedding_dim': 300, 'model.entity_initializer': 'xavier_uniform', 'loss.margin': 4.0, 'regularizer.weight': 1e-05, 'regularizer.p': 2, 'optimizer.lr': 0.01, 'negative_sampler.num_negs_per_pos': 12}
{'model.embedding_dim': 300, 'model.relation_dim': 112, 'model.dropout_0': 0.1, 'model.dropout_1': 0.2, 'model.dropout_2': 0.1, 'model.entity_initializer': 'xavier_uniform', 'optimizer.lr': 0.001, 'negative_sampler.num_negs_per_pos': 14}


# Training loops

## TransE

In [6]:
#Specifying all hyperparameters

regularizer=LpRegularizer(
    p=TransE_best_params['regularizer.p'],
    weight = TransE_best_params['regularizer.weight']
)

loss = MarginRankingLoss(
    margin=TransE_best_params['loss.margin']
)

model=TransE(
    triples_factory = train,
    embedding_dim=TransE_best_params['model.embedding_dim'],
    scoring_fct_norm=TransE_best_params['model.scoring_fct_norm'],
    entity_initializer=TransE_best_params['model.entity_initializer'],
    regularizer=regularizer,
    loss=loss,
    random_seed=42
)

stopper = EarlyStopper(
    model=model,
    evaluator=RankBasedEvaluator(filtered=True),      
    training_triples_factory=train,                   
    evaluation_triples_factory=val, 
    frequency=25,              
    patience=3,               
    relative_delta=0.001,   
    metric='mean_reciprocal_rank',
    larger_is_better=True    
)

negative_sampler=BernoulliNegativeSampler(
    mapped_triples=train.mapped_triples,
    filtered=True,
    num_negs_per_pos=TransE_best_params['negative_sampler.num_negs_per_pos']
)

optimizer=Adam(
    model.parameters(),
    lr=TransE_best_params['optimizer.lr']
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

In [7]:
#Specifying the Training loop

training_loop = SLCWATrainingLoop(
    triples_factory = train,
    model = model,
    negative_sampler=negative_sampler,
    optimizer = optimizer
)

_ = training_loop.train(
    triples_factory=train,
    num_epochs=200,
    batch_size=512,
    stopper=stopper,
    checkpoint_name='TransE_checkpoint.pt',
    checkpoint_frequency=0,
    checkpoint_directory='/kaggle/working'
)

losses={'loss':_}
with open('losses_TransE.json', 'w') as f:
    json.dump(losses, f)

save_model(model, "TransE.pt")
print('****** MODEL SAVED ******')

evaluator = RankBasedEvaluator(
    filtered=True
    )

result = evaluator.evaluate(
    model = model,
    mapped_triples=test.mapped_triples,
    batch_size=256,
    additional_filter_triples=[
        train.mapped_triples,
        val.mapped_triples
    ]
)


print(f"Hits@1: \t\t{result.get_metric('hits@1'): 5.3f}")
print(f"Hits@3: \t\t{result.get_metric('hits@3'): 5.3f}")
print(f"Hits@5: \t\t{result.get_metric('hits@5'): 5.3f}")
print(f"Hits@10: \t\t{result.get_metric('hits@10'): 5.3f}")
print(f"Mean Reciprocal Rank:  \t{result.get_metric('mean_reciprocal_rank'): 5.3f}")
print(f"Mean Rank:  \t\t{result.get_metric('mean_rank'): 5.3f}")

with open('results_TransE.json', 'w') as f:
    json.dump(result.to_dict(), f)

print('****** RESULTS SAVED ******')

Training epochs on cuda:0:   0%|          | 0/200 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

****** MODEL SAVED ******


Evaluating on cuda:0:   0%|          | 0.00/102k [00:00<?, ?triple/s]

Hits@1: 		 0.496
Hits@3: 		 0.580
Hits@5: 		 0.608
Hits@10: 		 0.641
Mean Reciprocal Rank:  	 0.548
Mean Rank:  		 2050.949
****** RESULTS SAVED ******


## RotatE

In [8]:
#Specifying all hyperparameters

regularizer=LpRegularizer(
    p=RotatE_best_params['regularizer.p'],
    weight = RotatE_best_params['regularizer.weight']
)

loss = MarginRankingLoss(
    margin=RotatE_best_params['loss.margin']
)

model=RotatE(
    triples_factory = train,
    embedding_dim=RotatE_best_params['model.embedding_dim'],
    entity_initializer=RotatE_best_params['model.entity_initializer'],
    regularizer=regularizer,
    loss=loss,
    random_seed=42
)

stopper = EarlyStopper(
    model=model,
    evaluator=RankBasedEvaluator(filtered=True),      
    training_triples_factory=train,                   
    evaluation_triples_factory=val, 
    frequency=25,              
    patience=3,               
    relative_delta=0.001,   
    metric='mean_reciprocal_rank',
    larger_is_better=True    
)

negative_sampler=BernoulliNegativeSampler(
    mapped_triples=train.mapped_triples,
    filtered=True,
    num_negs_per_pos=RotatE_best_params['negative_sampler.num_negs_per_pos']
)

optimizer=Adam(
    model.parameters(),
    lr=RotatE_best_params['optimizer.lr']
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

In [9]:
#Specifying the Training loop

training_loop = SLCWATrainingLoop(
    triples_factory = train,
    model = model,
    negative_sampler=negative_sampler,
    optimizer = optimizer
)

_ = training_loop.train(
    triples_factory=train,
    num_epochs=200,
    batch_size=512,
    stopper=stopper,
    checkpoint_name='RotatE_checkpoint.pt',
    checkpoint_frequency=0,
    checkpoint_directory='/kaggle/working'
)

losses={'loss':_}
with open('losses_RotatE.json', 'w') as f:
    json.dump(losses, f)

save_model(model, "RotatE.pt")
print('****** MODEL SAVED ******')

evaluator = RankBasedEvaluator(
    filtered=True
    )

result = evaluator.evaluate(
    model = model,
    mapped_triples=test.mapped_triples,
    batch_size=256,
    additional_filter_triples=[
        train.mapped_triples,
        val.mapped_triples
    ]
)


print(f"Hits@1: \t\t{result.get_metric('hits@1'): 5.3f}")
print(f"Hits@3: \t\t{result.get_metric('hits@3'): 5.3f}")
print(f"Hits@5: \t\t{result.get_metric('hits@5'): 5.3f}")
print(f"Hits@10: \t\t{result.get_metric('hits@10'): 5.3f}")
print(f"Mean Reciprocal Rank:  \t{result.get_metric('mean_reciprocal_rank'): 5.3f}")
print(f"Mean Rank:  \t\t{result.get_metric('mean_rank'): 5.3f}")

with open('results_RotatE.json', 'w') as f:
    json.dump(result.to_dict(), f)

print('****** RESULTS SAVED ******')

Training epochs on cuda:0:   0%|          | 0/200 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/641 [00:00<?, ?batch/s]

****** MODEL SAVED ******


Evaluating on cuda:0:   0%|          | 0.00/102k [00:00<?, ?triple/s]

Hits@1: 		 0.474
Hits@3: 		 0.553
Hits@5: 		 0.585
Hits@10: 		 0.638
Mean Reciprocal Rank:  	 0.534
Mean Rank:  		 177.388
****** RESULTS SAVED ******


## TuckER

In [7]:
#Specifying all hyperparameters

'''regularizer=LpRegularizer(
    p=2,
    weight = 0.01
)'''

loss = BCEAfterSigmoidLoss()

model=TuckER(
    triples_factory = train,
    embedding_dim=Tucker_best_params['model.embedding_dim'],
    relation_dim=Tucker_best_params['model.relation_dim'],
    dropout_0=Tucker_best_params['model.dropout_0'],
    dropout_1=Tucker_best_params['model.dropout_1'],
    dropout_2=Tucker_best_params['model.dropout_2'],
    entity_initializer=Tucker_best_params['model.entity_initializer'],
    #regularizer=regularizer,
    loss=loss,
    random_seed=42
)

stopper = EarlyStopper(
    model=model,
    evaluator=RankBasedEvaluator(filtered=True),      
    training_triples_factory=train,                   
    evaluation_triples_factory=val, 
    frequency=25,              
    patience=3,               
    relative_delta=0.001,   
    metric='mean_reciprocal_rank',
    larger_is_better=True    
)

negative_sampler=BernoulliNegativeSampler(
    mapped_triples=train.mapped_triples,
    filtered=True,
    num_negs_per_pos=Tucker_best_params['negative_sampler.num_negs_per_pos']
)

optimizer=Adam(
    model.parameters(),
    lr=Tucker_best_params['optimizer.lr']
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)

In [8]:
#Specifying the Training loop

training_loop = SLCWATrainingLoop(
    triples_factory = train,
    model = model,
    negative_sampler=negative_sampler,
    optimizer = optimizer
)

_ = training_loop.train(
    triples_factory=train,
    num_epochs=200,
    batch_size=512,
    stopper=stopper,
    checkpoint_name='TuckER_checkpoint.pt',
    checkpoint_frequency=5,
    checkpoint_directory='/kaggle/working/checkpoints'
)

losses={'loss':_}
with open('losses_TuckER.json', 'w') as f:
    json.dump(losses, f)

save_model(model, "TuckER.pt")
print('****** MODEL SAVED ******')

evaluator = RankBasedEvaluator(
    filtered=True
    )

result = evaluator.evaluate(
    model = model,
    mapped_triples=test.mapped_triples,
    batch_size=256,
    additional_filter_triples=[
        train.mapped_triples,
        val.mapped_triples
    ]
)


print(f"Hits@1: \t\t{result.get_metric('hits@1'): 5.3f}")
print(f"Hits@3: \t\t{result.get_metric('hits@3'): 5.3f}")
print(f"Hits@5: \t\t{result.get_metric('hits@5'): 5.3f}")
print(f"Hits@10: \t\t{result.get_metric('hits@10'): 5.3f}")
print(f"Mean Reciprocal Rank:  \t{result.get_metric('mean_reciprocal_rank'): 5.3f}")
print(f"Mean Rank:  \t\t{result.get_metric('mean_rank'): 5.3f}")

with open('results_TuckER.json', 'w') as f:
    json.dump(result.to_dict(), f)

print('****** RESULTS SAVED ******')

Training epochs on cuda:0:   0%|          | 0/200 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/640 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/640 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/640 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/640 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/640 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/640 [00:00<?, ?batch/s]

****** MODEL SAVED ******


Evaluating on cuda:0:   0%|          | 0.00/102k [00:00<?, ?triple/s]

Hits@1: 		 0.567
Hits@3: 		 0.674
Hits@5: 		 0.720
Hits@10: 		 0.785
Mean Reciprocal Rank:  	 0.642
Mean Rank:  		 100.324
****** RESULTS SAVED ******


### Saving entities and relationship Ids

In [9]:
with open('entity_to_id.json', 'w') as f:
    json.dump(data.entity_to_id, f)
with open('relation_to_id.json', 'w') as f:
    json.dump(data.relation_to_id, f)